<div style="border-radius:10px;
            border: #0b0265 solid;
           background-color:#e8efff;
           font-size:110%;
           letter-spacing:0.5px;
            text-align: center">

<center><h1 style="padding: 25px 0px; color:#0b0265; font-weight: bold; font-family: Cursive">
Titanic 🚢</h1></center>
<center><h2 style="padding-bottom: 25px; color:#0b0265; font-weight: bold; font-style:italic; font-family: Cursive">
(With Ensemble algorithms)</h2></center>     

</div>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Libraries

In [ ]:
import numpy as np 
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import auc
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.linear_model import LassoCV, Ridge, ElasticNet, LogisticRegressionCV

from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier

from imblearn.over_sampling import SMOTE

from statsmodels.stats.outliers_influence import variance_inflation_factor

import warnings
warnings.filterwarnings("ignore")

# Define Functions

In [ ]:
def checkVIF(X):
    vif = pd.DataFrame()
    vif['Features'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False,)
    return(vif)

# ------------------------------------------------------------------
def Cls_model_RndSrch_Tune(model, Data, X, y, params):
    
    clf = RandomizedSearchCV(model, params, scoring ='accuracy', cv = 5, n_jobs=-1, random_state=100)
    clf.fit(X, y)
    
    print("best score is :" , clf.best_score_)
    print("best estimator is :" , clf.best_estimator_)
    print("best Params is :" , clf.best_params_)
    
    return (clf.best_score_)

# Data Understanding

In [ ]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

In [ ]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

In [ ]:
print(train_data.shape)
print(test_data.shape)

In [ ]:
train_data.columns

In [ ]:
train_data.info()

In [ ]:
test_data.info()

In [ ]:
sns.countplot(x = "Survived", data = train_data)
train_data.loc[:, 'Survived'].value_counts()

In [ ]:
train_data.describe().T

In [ ]:
p = train_data.hist(figsize = (20,20))

# Preproccessing

In [ ]:
plt.figure(figsize=(12,10), dpi= 80)
ax = sns.boxplot(data=train_data, orient="h", palette="husl", whis=1.5)

In [ ]:
train_data.isnull().sum()

In [ ]:
train_data.drop(['Cabin'], axis=1, inplace=True)
test_data.drop(['Cabin'], axis=1, inplace=True)
train_data.head()

In [ ]:
train_data.columns

In [ ]:
print("Mode: {}".format(train_data.Age.mode()))
print("Mean: {}".format(train_data.Age.mean()))
print("Median: {}".format(train_data.Age.median()))
print("25%: {}".format(train_data.Age.quantile(0.25)))
print("75%: {}".format(train_data.Age.quantile(0.75)))
print("10%: {}".format(train_data.Age.quantile(0.10)))
print("90%: {}".format(train_data.Age.quantile(0.90)))
print("min: {}".format(train_data.Age.min()))
print("max: {}".format(train_data.Age.max()))

Q1 = train_data.quantile(0.25)
Q3 = train_data.quantile(0.75)
IQR = Q3 - Q1
# print(IQR)

print("lower: {}".format(Q1['Age']-(1.5*IQR['Age'])))
print("upper: {}".format(Q3['Age']+(1.5*IQR['Age'])))

In [ ]:
train_data['Age'] = train_data['Age'].fillna(train_data.Age.median())
test_data['Age'] = test_data['Age'].fillna(test_data.Age.median())

In [ ]:
train_data = train_data.dropna(how='any', axis= 0)

In [ ]:
print("Mode: {}".format(test_data.Fare.mode()))
print("Mean: {}".format(test_data.Fare.mean()))
print("Median: {}".format(test_data.Fare.median()))
print("25%: {}".format(test_data.Fare.quantile(0.25)))
print("75%: {}".format(test_data.Fare.quantile(0.75)))
print("10%: {}".format(test_data.Fare.quantile(0.10)))
print("90%: {}".format(test_data.Fare.quantile(0.90)))
print("min: {}".format(test_data.Fare.min()))
print("max: {}".format(test_data.Fare.max()))

Q1 = test_data.quantile(0.25)
Q3 = test_data.quantile(0.75)
IQR = Q3 - Q1
# print(IQR)

print("lower: {}".format(Q1['Fare']-(1.5*IQR['Fare'])))
print("upper: {}".format(Q3['Fare']+(1.5*IQR['Fare'])))

In [ ]:
test_data['Fare'] = test_data['Fare'].fillna(test_data.Fare.median())

In [ ]:
train_data.isnull().sum()

In [ ]:
test_data.isnull().sum()

In [ ]:
train_data.loc[train_data.duplicated()].shape

In [ ]:
test_data.loc[test_data.duplicated()].shape

In [ ]:
train_data.columns

In [ ]:
train_data.drop(['Name','Ticket'], axis=1, inplace=True)
test_data.drop(['Name','Ticket'], axis=1, inplace=True)
train_data.head()

In [ ]:
mask = np.triu(np.ones_like(train_data.corr()))
fig, ax = plt.subplots(figsize=(30,25),dpi=80, facecolor='w', edgecolor='k')
sns.heatmap(train_data.corr(), mask= mask, cmap="YlGnBu", vmax=.3, annot = True, center = 0,annot_kws={"fontsize":22})

In [ ]:
train_data.columns

In [ ]:
categorical_feature = ['Sex','Embarked']

In [ ]:
le = LabelEncoder() 
train_data[categorical_feature] = train_data[categorical_feature].apply(lambda col: le.fit_transform(col)) 
test_data[categorical_feature] = test_data[categorical_feature].apply(lambda col: le.fit_transform(col)) 
train_data.head()

In [ ]:
x = train_data.drop("Survived", axis = 1)
y = train_data['Survived']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 100,stratify=y, test_size = 0.2)
print(y_train.value_counts())

In [ ]:
scale = MinMaxScaler()

col = ['Pclass', 'Age', 'SibSp', 'Parch','Fare', 'Embarked']

x_train[col] = scale.fit_transform(x_train[col])
x_train.head()

In [ ]:
x_test[col] = scale.transform(x_test[col])
test_data[col] = scale.transform(test_data[col])
x_test.head()

# Manage Imbalance Data

In [ ]:
smt = SMOTE(random_state=100)
x_train, y_train = smt.fit_resample(x_train, y_train)
np.bincount(y_train)

# Feature Selection

In [ ]:
checkVIF(x).reset_index(drop=True)

In [ ]:
dt=DecisionTreeClassifier()
dt.fit(x_train,y_train)
feat_importances1 = pd.Series(dt.feature_importances_, index=x_train.columns)
feat_importances1.sort_values(ascending=True).plot(kind='barh')
plt.show()

In [ ]:
rf = RandomForestClassifier()
rf.fit(x_train,y_train)
feat_importances1 = pd.Series(rf.feature_importances_, index=x_train.columns)
feat_importances1.sort_values(ascending=True).plot(kind='barh')
plt.show()

In [ ]:
gb = GradientBoostingClassifier()
gb.fit(x_train,y_train)
feat_importances1 = pd.Series(gb.feature_importances_, index=x_train.columns)
feat_importances1.sort_values(ascending=True).plot(kind='barh')
plt.show()

In [ ]:
et = ExtraTreesClassifier()
et.fit(x_train,y_train)
feat_importances1 = pd.Series(et.feature_importances_, index=x_train.columns)
feat_importances1.sort_values(ascending=True).plot(kind='barh')
plt.show()

In [ ]:
#alphas = [0.0001, 0.001, 0.01, 0.1]
ls = LassoCV(max_iter=100, random_state=100, n_jobs=-1)
clf = ls.fit(x_train, y_train)
importance = np.abs(clf.coef_)
frame = pd.Series(importance, x_train.columns)
frame.sort_values(ascending = True).plot(kind='barh')
plt.show()

In [ ]:
en = ElasticNet(max_iter=100, alpha=0.0001, random_state=100, selection='cyclic')
clf = en.fit(x_train, y_train)
importance = np.abs(clf.coef_)
frame = pd.Series(importance, x_train.columns)
frame.sort_values(ascending = True).plot(kind='barh')
plt.show()

In [ ]:
r = Ridge(max_iter=100, random_state=100, normalize=True)
clf = r.fit(x_train, y_train)
importance = np.abs(clf.coef_)
frame = pd.Series(importance, x_train.columns)
frame.sort_values(ascending = True).plot(kind='barh', )
plt.title("Feature Selection with Ridge")
plt.show()

In [ ]:
correlation=(train_data.corr()['Survived'])
del correlation['Survived']
correlation.sort_values(ascending=True).plot(kind='barh',color = "darkgreen")
plt.title("correlation")
plt.show()

In [ ]:
# new Feaures
x_train_new = x_train[['Pclass', 'Sex', 'Age', 'Fare','Embarked']]
x_test_new = x_test[['Pclass', 'Sex', 'Age', 'Fare','Embarked']]

# Hyperparameter Tuning and Build Models

In [ ]:
param_DT = {'criterion':['gini', 'entropy'] ,
            'max_depth':[None,3,4,5] ,
            'min_samples_split':[2,3,4,5,6] ,
            'min_samples_leaf':[2,3,4,5,6]}

Cls_model_RndSrch_Tune(DecisionTreeClassifier(random_state = 0), train_data, x_train_new, y_train, param_DT)

In [ ]:
param_RF = {'criterion':['gini', 'entropy'] ,
            'max_depth':[ None,3,4,5] ,
            'min_samples_split':[2,3,4,5,6] ,
            'min_samples_leaf':[2,3,4,5,6],
            'n_estimators':[25,50,75,100,200,300]}

Cls_model_RndSrch_Tune(RandomForestClassifier(random_state = 0), train_data, x_train_new, y_train, param_RF)

In [ ]:
param_ET = {'criterion':['gini', 'entropy'] ,
            'max_depth':[3,4,5] ,
            'min_samples_split':[2,3,4,5,6] ,
            'min_samples_leaf':[2,3,4,5,6],
            'n_estimators':[25,50,75,100,200,300],
            'max_features' : ['auto', 'sqrt', 'log2']}

Cls_model_RndSrch_Tune(ExtraTreesClassifier(random_state = 0), train_data, x_train_new, y_train, param_ET)

In [ ]:
param_MLP = {'activation' : ['relu' , 'logistic' , 'tanh'],
            'hidden_layer_sizes':[(10), (20), (20,30)],
            'max_iter' : [10, 50, 100, 200],
            'solver': ['sgd', 'adam'],
            'learning_rate_init': [0.01, 0.001, 0.0001, 0.025, 0.1, 0.2]}

Cls_model_RndSrch_Tune(MLPClassifier(random_state = 0), train_data, x_train_new, y_train, param_MLP)

In [ ]:
param_SVM = {'kernel' : ['linear', 'rbf', 'poly'],
            'gamma' : [0.01, 0.1, 0.2, 0.3, 0.4,0.5, 0.6,0.7], 
            'C':  [0.01 , 0.1, 1, 10, 100],
            'degree': [2, 3, 4, 5, 6]}


Cls_model_RndSrch_Tune(SVC(random_state = 0), train_data, x_train_new, y_train, param_SVM)

In [ ]:
param_GB = {'loss' : ['deviance', 'exponential'],
            'n_estimators':[25,50,75,100,200,300],
            'criterion' : ['friedman_mse', 'mse', 'mae'],
            'min_samples_split':[2,3,4,5,6] ,
            'min_samples_leaf':[2,3,4,5,6],
            'max_depth':[3,4,5]}

Cls_model_RndSrch_Tune(GradientBoostingClassifier(random_state = 0), train_data, x_train_new, y_train, param_GB)

In [ ]:
param_LR = {'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
            'C': range(1,11),
            'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
            'multi_class' : ['auto', 'ovr', 'multinomial']}


Cls_model_RndSrch_Tune(LogisticRegression(random_state = 0), train_data, x_train_new, y_train, param_LR)

<div style="border-radius:10px;
            background-color:#ffffff;
            border-style: solid;
            border-color: #0b0265;
            letter-spacing:0.5px;">

<center><h3 style="padding: 5px 0px; color:#0b0265; font-weight: bold; font-family: Cursive">
Models</h3></center>
</div>

In [ ]:
DT = DecisionTreeClassifier(max_depth=5, min_samples_leaf=5, min_samples_split=4,
                       random_state=0)

DT.fit(x_train_new, y_train)

y_test_pred_DT = DT.predict(x_test_new)
y_train_pred_DT = DT.predict(x_train_new)

test_acc_DT = accuracy_score(y_test, y_test_pred_DT)
train_acc_DT = accuracy_score(y_train, y_train_pred_DT)

precision_score_DT = precision_score(y_test, y_test_pred_DT)
recall_score_DT = recall_score(y_test, y_test_pred_DT)
f1_score_DT = f1_score(y_test, y_test_pred_DT)

print("Tain set Accuracy: ", train_acc_DT)
print("Test set Accuracy: ", test_acc_DT)
print("************************************************")
print("precision_score: ", precision_score_DT)
print("recall_score: ", recall_score_DT)
print("f1_score: ", f1_score_DT)

In [ ]:
MLP = MLPClassifier(hidden_layer_sizes=(20, 30), learning_rate_init=0.1,
              random_state=0, solver='sgd')

MLP.fit(x_train_new, y_train)

y_test_pred_MLP = MLP.predict(x_test_new)
y_train_pred_MLP = MLP.predict(x_train_new)

test_acc_MLP = accuracy_score(y_test, y_test_pred_MLP)
train_acc_MLP = accuracy_score(y_train, y_train_pred_MLP)

precision_score_MLP = precision_score(y_test, y_test_pred_MLP)
recall_score_MLP = recall_score(y_test, y_test_pred_MLP)
f1_score_MLP = f1_score(y_test, y_test_pred_MLP)
conf_MLP = confusion_matrix(y_test, y_test_pred_MLP)

print("Tain set Accuracy: ", train_acc_MLP)
print("Test set Accuracy: ", test_acc_MLP)
print("************************************************")
print("precision_score: ", precision_score_MLP)
print("recall_score: ", recall_score_MLP)
print("f1_score: ", f1_score_MLP)
print("************************************************")

In [ ]:
RF = RandomForestClassifier(criterion='entropy', max_depth=5, min_samples_leaf=4,
                       min_samples_split=3, n_estimators=50, random_state=0)

RF.fit(x_train_new, y_train)

y_test_pred_RF = RF.predict(x_test_new)
y_train_pred_RF = RF.predict(x_train_new)

test_acc_RF = accuracy_score(y_test, y_test_pred_RF)
train_acc_RF = accuracy_score(y_train, y_train_pred_RF)

precision_score_RF = precision_score(y_test, y_test_pred_RF)
recall_score_RF = recall_score(y_test, y_test_pred_RF)
f1_score_RF = f1_score(y_test, y_test_pred_RF)
conf_RF = confusion_matrix(y_test, y_test_pred_RF)

print("Tain set Accuracy: ", train_acc_RF)
print("Test set Accuracy: ", test_acc_RF)
print("************************************************")
print("precision_score: ", precision_score_RF)
print("recall_score: ", recall_score_RF)
print("f1_score: ", f1_score_RF)
print("************************************************")

In [ ]:
GB = GradientBoostingClassifier(criterion='mse', loss='exponential', max_depth=3,
                           min_samples_leaf=3, min_samples_split=4,
                           n_estimators=25, random_state=0)

GB.fit(x_train_new, y_train)

y_test_pred_GB = GB.predict(x_test_new)
y_train_pred_GB = GB.predict(x_train_new)

test_acc_GB = accuracy_score(y_test, y_test_pred_GB)
train_acc_GB = accuracy_score(y_train, y_train_pred_GB)

precision_score_GB = precision_score(y_test, y_test_pred_GB)
recall_score_GB = recall_score(y_test, y_test_pred_GB)
f1_score_GB = f1_score(y_test, y_test_pred_GB)
conf_GB = confusion_matrix(y_test, y_test_pred_GB)

print("Tain set Accuracy: ", train_acc_GB)
print("Test set Accuracy: ", test_acc_GB)
print("************************************************")
print("precision_score: ", precision_score_GB)
print("recall_score: ", recall_score_GB)
print("f1_score: ", f1_score_GB)
print("************************************************")

In [ ]:
SVM = SVC(C=100, degree=5, gamma=0.7, kernel='poly', random_state=0, probability=True)

SVM.fit(x_train_new, y_train)

y_test_pred_SVM = SVM.predict(x_test_new)
y_train_pred_SVM = SVM.predict(x_train_new)

test_acc_SVM = accuracy_score(y_test, y_test_pred_SVM)
train_acc_SVM = accuracy_score(y_train, y_train_pred_SVM)

precision_score_SVM = precision_score(y_test, y_test_pred_SVM)
recall_score_SVM = recall_score(y_test, y_test_pred_SVM)
f1_score_SVM = f1_score(y_test, y_test_pred_SVM)
conf_SVM = confusion_matrix(y_test, y_test_pred_SVM)

roc_test_SVM = roc_auc_score(y_test, y_test_pred_SVM)
roc_train_SVM = roc_auc_score(y_train, y_train_pred_SVM)


print("Tain set Accuracy: ", train_acc_SVM)
print("Test set Accuracy: ", test_acc_SVM)
print("************************************************")
print("precision_score: ", precision_score_SVM)
print("recall_score: ", recall_score_SVM)
print("f1_score: ", f1_score_SVM)
print("************************************************")

In [ ]:
ET = ExtraTreesClassifier(criterion='entropy', max_depth=3, max_features='sqrt',
                     min_samples_leaf=3, min_samples_split=4, n_estimators=75,
                     random_state=0)
ET.fit(x_train_new, y_train)

y_test_pred_ET = ET.predict(x_test_new)
y_train_pred_ET = ET.predict(x_train_new)

test_acc_ET = accuracy_score(y_test, y_test_pred_ET)
train_acc_ET = accuracy_score(y_train, y_train_pred_ET)

precision_score_ET = precision_score(y_test, y_test_pred_ET)
recall_score_ET = recall_score(y_test, y_test_pred_ET)
f1_score_ET = f1_score(y_test, y_test_pred_ET)
conf_ET = confusion_matrix(y_test, y_test_pred_ET)

print("Tain set Accuracy: ", train_acc_ET)
print("Test set Accuracy: ", test_acc_ET)
print("************************************************")
print("precision_score: ", precision_score_ET)
print("recall_score: ", recall_score_ET)
print("f1_score: ", f1_score_ET)
print("************************************************")

In [ ]:
LR = LogisticRegression(C=7, random_state=0, solver='newton-cg')
LR.fit(x_train_new, y_train)

y_test_pred_LR = LR.predict(x_test_new)
y_train_pred_LR = LR.predict(x_train_new)

test_acc_LR = accuracy_score(y_test, y_test_pred_LR)
train_acc_LR = accuracy_score(y_train, y_train_pred_LR)

precision_score_LR = precision_score(y_test, y_test_pred_LR)
recall_score_LR = recall_score(y_test, y_test_pred_LR)
f1_score_LR = f1_score(y_test, y_test_pred_LR)
conf_LR = confusion_matrix(y_test, y_test_pred_LR)

print("Tain set Accuracy: ", train_acc_LR)
print("Test set Accuracy: ", test_acc_LR)
print("************************************************")
print("precision_score: ", precision_score_LR)
print("recall_score: ", recall_score_LR)
print("f1_score: ", f1_score_LR)
print("************************************************")

# Compare Models 

In [ ]:
models = [('Decision Tree', train_acc_DT, test_acc_DT, precision_score_DT, recall_score_DT, f1_score_DT, 'Good'),
          ('Random Forest', train_acc_RF, test_acc_RF, precision_score_RF, recall_score_RF, f1_score_RF, 'Good'),
          ('Neural Network', train_acc_MLP, test_acc_MLP, precision_score_MLP, recall_score_MLP, f1_score_MLP,'Good'),
          ('SVC', train_acc_SVM, test_acc_SVM, precision_score_SVM, recall_score_SVM, f1_score_SVM,'Good'),
          ('Extra Tree', train_acc_ET, test_acc_ET, precision_score_ET, recall_score_ET, f1_score_ET,'Good'),          
          ('Gradient Boosting', train_acc_GB, test_acc_GB, precision_score_GB, recall_score_GB, f1_score_GB,'*** The Best ***'),
          
         ]

In [ ]:
predict_IR = pd.DataFrame(data = models, columns=['Model', 'Train_accuracy', 'Test_accuracy','precision_score', 'recall_score', 'f1_score', 'Description'])
predict_IR.style.background_gradient(cmap='YlGn')

# Prediction

In [ ]:
test_data.shape

In [ ]:
# new
test_data_new = test_data[['Pclass', 'Sex', 'Age', 'Fare','Embarked']]

In [ ]:
predictions = GB.predict(test_data_new)
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

<div style="border-radius:10px;
            background-color:#ffffff;
            border-style:solid;
            border-color: #0b0265;
            letter-spacing:0.5px;">

<center><h4 style="padding: 5px 0px; color:#0b0265; font-weight: bold; font-family: Cursive">
    Thanks for your attention and for reviewing my notebook.🙌 <br><br>Please write your comments for me.📝</h4></center>
<center><h4 style="padding: 5px 0px; color:#0b0265; font-weight: bold; font-family: Cursive">
If you liked my work and found it useful, please upvote. Thank you🙏</h4></center>
</div>